[View in Colaboratory](https://colab.research.google.com/github/Masum06/gender_newspaper/blob/master/newspaper_word2vec_gensim_batch.ipynb)

**Description**



*   Install Gensim and Download Newspaper analyzed data
*   Train Gensim word2vec with the selected Newspaper data
*   Save trained model in batch



### 1. Connect to Google drive

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
#@title Default title text
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
ls -l drive/data/newspaper_text/

total 998847
-rw-r--r-- 1 root root 227817153 Aug  9 02:36 bothReplaced.txt
-rw-r--r-- 1 root root  16748894 Aug  9 02:44 both_set.pickle
-rw-r--r-- 1 root root 233148951 Aug  9 02:36 femaleReplaced.txt
-rw-r--r-- 1 root root  16253484 Aug  9 02:44 intersection.pickle
-rw-r--r-- 1 root root 228892288 Aug  9 02:36 maleReplaced.txt
-rw-r--r-- 1 root root  18046887 Aug  9 02:44 man_set.pickle
-rw-r--r-- 1 root root  22017202 Aug  9 02:44 none_set.pickle
-rw-r--r-- 1 root root 234264261 Aug  8 03:02 nonGender.txt
-rw-r--r-- 1 root root   4885118 Aug  9 02:42 uniqueNames.txt
-rw-r--r-- 1 root root  20741813 Aug  9 02:44 woman_set.pickle


In [0]:
!cp drive/data/newspaper_text/*.txt ./

In [5]:
ls -l

total 2079136
-rw-r--r-- 1 root root 227817153 Aug 14 08:49 bothReplaced.txt
drwxr-xr-x 1 root root      4096 Aug 14 08:48 datalab/
drwxr-xr-x 2 root root      4096 Aug 14 08:49 drive/
-rw-r--r-- 1 root root 233148951 Aug 14 08:49 femaleReplaced.txt
drwxr-xr-x 2 root root      4096 Aug 14 06:30 glove/
-rw-r--r-- 1 root root 862182613 Aug 14 06:30 glove.6B.zip
-rw-r--r-- 1 root root 228892288 Aug 14 08:50 maleReplaced.txt
drwxr-xr-x 8 root root      4096 Aug 14 06:29 MITIE/
-rw-r--r-- 1 root root 337804218 Aug 14 06:46 my_text_categorizer.dat
-rw-r--r-- 1 root root 234264261 Aug 14 08:50 nonGender.txt
-rw-r--r-- 1 root root   4885118 Aug 14 08:50 uniqueNames.txt


### 2. From Mega

In [0]:
%%bash
git clone https://github.com/jeroenmeulenaar/python3-mega.git python3mega
cd python3mega
pip install -r requirements.txt

  Running setup.py bdist_wheel for URLObject: started
  Running setup.py bdist_wheel for URLObject: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/fd/7e/18/ccb55ecc2834f945b769c1ff1df12ca5a14400ccfc58e3c515
  Running setup.py bdist_wheel for pycrypto: started
  Running setup.py bdist_wheel for pycrypto: finished with status 'done'
  Stored in directory: /content/.cache/pip/wheels/27/02/5e/77a69d0c16bb63c6ed32f5386f33a2809c94bd5414a2f6c196
Successfully built URLObject pycrypto


Cloning into 'python3mega'...


In [0]:
import os
os.chdir("python3mega")
from mega import Mega
m = Mega.from_ephemeral()
os.chdir("..")
#from python3mega.mega import Mega

In [0]:
m.download_from_url('https://mega.nz/#!NMwSGSCJ!K5L_g9uioqMP2oXzHRmLRUItHFMuYk7PjPfka1P7_ek') #news_db.json

In [0]:
ls

datalab/  news_db.json  python3mega/


In [0]:
!unzip newspaper_text.zip

In [0]:
!mv newspaper_text/*.txt ./

###Installation

In [6]:
!pip install gensim

    100% |████████████████████████████████| 23.5MB 1.6MB/s 
    100% |████████████████████████████████| 1.4MB 12.9MB/s 
    100% |████████████████████████████████| 133kB 22.3MB/s 
    100% |████████████████████████████████| 4.4MB 6.9MB/s 
    100% |████████████████████████████████| 61kB 19.7MB/s 
    100% |████████████████████████████████| 552kB 20.5MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


###Data preprocessing

In [0]:
import numpy as np
import os
from random import shuffle
import re
import sys
import json
from gensim.models import Word2Vec

In [0]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

def diff(t_a, t_b):
    t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
    return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)

In [4]:
files = ['nonGender', 'bothReplaced'] #'maleReplaced', 'femaleReplaced', 

model = "to be deleted"

for data in files:
  
  # CREATING THE DATASET
  with open(data+".txt", 'r') as myfile:
      input_text=myfile.read().replace('\n', '')

  # remove parenthesis 
  input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)
  # store as list of sentences
  sentences_strings = []
  for line in input_text_noparens.split('\n'):
      m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
      sentences_strings.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)
  # store as list of lists of words
  sentences = []
  for sent_str in sentences_strings:
      tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
      sentences.append(tokens)
      
  # BUILDING 5 MODELS AT A TIME
  for i in range(5):
    
    del model

    t_a = datetime.now()
    
    model = Word2Vec(sentences=sentences, size=100, window=10, min_count=0, workers=4, sg=0)
    model_name = data+"_model_"+str(i)
    model.save(model_name)
    !cp $model_name drive/data/models/
    
    t_b = datetime.now()

    print("Training time for ", model_name, diff(t_a, t_b))


Training time for  nonGender_model_0 0h 5m 48s
Training time for  nonGender_model_1 0h 5m 48s
Training time for  nonGender_model_2 0h 5m 48s
Training time for  nonGender_model_3 0h 5m 49s
Training time for  nonGender_model_4 0h 5m 48s
Training time for  bothReplaced_model_0 0h 5m 29s
Training time for  bothReplaced_model_1 0h 5m 31s
Training time for  bothReplaced_model_2 0h 5m 31s
Training time for  bothReplaced_model_3 0h 5m 31s
Training time for  bothReplaced_model_4 0h 5m 28s


In [5]:
ls

bothReplaced_model_0
bothReplaced_model_0.trainables.syn1neg.npy
bothReplaced_model_0.wv.vectors.npy
bothReplaced_model_1
bothReplaced_model_1.trainables.syn1neg.npy
bothReplaced_model_1.wv.vectors.npy
bothReplaced_model_2
bothReplaced_model_2.trainables.syn1neg.npy
bothReplaced_model_2.wv.vectors.npy
bothReplaced_model_3
bothReplaced_model_3.trainables.syn1neg.npy
bothReplaced_model_3.wv.vectors.npy
bothReplaced_model_4
bothReplaced_model_4.trainables.syn1neg.npy
bothReplaced_model_4.wv.vectors.npy
bothReplaced.txt
datalab/
drive/
femaleReplaced_model_0
femaleReplaced_model_0.trainables.syn1neg.npy
femaleReplaced_model_0.wv.vectors.npy
femaleReplaced_model_1
femaleReplaced_model_1.trainables.syn1neg.npy
femaleReplaced_model_1.wv.vectors.npy
femaleReplaced_model_2
femaleReplaced_model_2.trainables.syn1neg.npy
femaleReplaced_model_2.wv.vectors.npy
femaleReplaced_model_3
femaleReplaced_model_3.trainables.syn1neg.npy
femaleReplaced_model_3.wv.vectors.npy
femaleReplaced_model_4
femaleRepla

https://radimrehurek.com/gensim/models/word2vec.html

[Tutorial](https://rare-technologies.com/word2vec-tutorial/)